# Test model example - HAY

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import shutil

import model
import evaluator
import plotting

# %matplotlib notebook
%matplotlib notebook

In [ ]:
probe_type = "linear"
model_name = "hay" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
protocols_file = "../data/hay_ecode_probe_planar/efeatures/protocols_BPO.json"
features_file = "../data/hay_ecode_probe_planar/efeatures/features_BPO.json"

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=features_file,
    protocols_file=protocols_file,
    electrode=probe,
    protocols_with_lfp=["IDrest_300"],
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
fitness_protocols
print(eva)

In [ ]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
responses

In [ ]:
plotting.plot_responses(responses)